<a href="https://colab.research.google.com/github/lphuong304/CS114.L21/blob/main/COLAB_ASSIGNMENTS/COLAB_ASSIGNMENT_17_05/Solution02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>BÀI TẬP COLAB SỐ 03 - 17/05/2021</b></h1>
<h1 align="center"><b>PREDICT FINAL SCORE OF IT001</b></h1>

###LỚP: CS114.L21
###NHÓM:
1. Nguyễn Ngọc Lan Phương - *19520227*
2. Hoàng Anh Tú - *19522450*
3. Nguyễn Quốc Huy - *19521623*

Nguồn tham khảo:




Link data:
* Data wecode: [link](https://raw.githubusercontent.com/lphuong304/CS114.L21/main/COLAB_ASSIGNMENTS/data%20IT001/wecode.csv)
* Data score: [link](https://raw.githubusercontent.com/lphuong304/CS114.L21/main/COLAB_ASSIGNMENTS/data%20IT001/wecode.csv)

* Ý tưởng:
Lọc các cột data không cần thiết, xử lý các cột dữ liệu lại theo hướng chỉ để lại một 


#1. IMPORT PACKAGE

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import  accuracy_score

**Check data**

In [ ]:
data_score = pd.read_csv('https://raw.githubusercontent.com/lphuong304/CS114.L21/main/COLAB_ASSIGNMENTS/data%20IT001/score_training.csv')
data = pd.read_csv('https://raw.githubusercontent.com/lphuong304/CS114.L21/main/COLAB_ASSIGNMENTS/data%20IT001/wecode.csv')

In [ ]:
data.head(10).T

,0,1,2,3,4,5,6,7,8,9
id,4,5,6,7,8,9,10,11,12,13
assignment_id,0,0,0,0,0,0,0,0,0,0
problem_id,3,4,5,5,5,5,6,7,8,9
user_id,4,4,4,4,4,4,4,4,4,4
is_final,1,1,0,0,0,0,1,1,1,1
status,SCORE,SCORE,Compilation Error,Compilation Error,Compilation Error,Compilation Error,SCORE,SCORE,SCORE,SCORE
pre_score,10000,10000,0,0,0,0,10000,10000,10000,10000
coefficient,100,100,100,100,100,100,100,100,100,100
file_name,solution-0,solution-1,solution-2,solution-3,solution-4,solution-5,solution-6,solution-7,solution-8,solution-9
language_id,2,2,2,2,2,2,2,2,2,2


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264606 entries, 0 to 264605
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             264606 non-null  int64 
 1   assignment_id  264606 non-null  int64 
 2   problem_id     264606 non-null  int64 
 3   user_id        264606 non-null  int64 
 4   is_final       264606 non-null  int64 
 5   status         264605 non-null  object
 6   pre_score      264606 non-null  int64 
 7   coefficient    264606 non-null  object
 8   file_name      264606 non-null  object
 9   language_id    264606 non-null  int64 
 10  created_at     264606 non-null  object
 11  updated_at     264606 non-null  object
dtypes: int64(7), object(5)
memory usage: 24.2+ MB


1. Một user-id submit nhiều lần ở 1 bài tập mới xanh
2. Một user-id submit ít lần(<=3) ở một bài tập rồi xanh
3. Solved/tries của mỗi user


#2. PREPROCESSING DATA

##2.1 Create new data frame:

remove column: id, created_at, updated_at, file_name, language_id


In [ ]:
data = data.drop(columns=['id', 'created_at', 'updated_at', 'file_name', 'language_id'])


In [ ]:
data.head(10).T

,0,1,2,3,4,5,6,7,8,9
assignment_id,0,0,0,0,0,0,0,0,0,0
problem_id,3,4,5,5,5,5,6,7,8,9
user_id,4,4,4,4,4,4,4,4,4,4
is_final,1,1,0,0,0,0,1,1,1,1
status,SCORE,SCORE,Compilation Error,Compilation Error,Compilation Error,Compilation Error,SCORE,SCORE,SCORE,SCORE
pre_score,10000,10000,0,0,0,0,10000,10000,10000,10000
coefficient,100,100,100,100,100,100,100,100,100,100


index = user_id

columns = ['tries'] tổng số lần submit của một user_id

In [ ]:
# Create a dataframe with index is user_id and data is number of  number of each user_id appear
tries = pd.DataFrame({'tries' : data['user_id'].value_counts()})
tries = tries.sort_index()
tries

,tries
1,2
4,54
5,6
6,130
7,5
...,...
1860,76
1862,12
1864,187
1865,99


index = user_id

lọc tìm những lần solved/user_id

In [ ]:
#@title
# Create dataframe[['user_id', 'is_final', 'pre_score']]
user_isfinal_score = data[['user_id', 'is_final', 'pre_score']]
user_isfinal_score

,user_id,is_final,pre_score
0,4,1,10000
1,4,1,10000
2,4,0,0
3,4,0,0
4,4,0,0
...,...,...,...
264601,1796,0,0
264602,1796,0,0
264603,1796,1,10000
264604,255,1,6250


In [ ]:
# Get data has 'is_final' = 1 and score = 10000 => solved problem
isfinal_submit = user_isfinal_score[user_isfinal_score['is_final'] > 0]
isfinal_submit = user_isfinal_score[user_isfinal_score['pre_score'] == 10000]
isfinal_submit = isfinal_submit.groupby('user_id').sum()
isfinal_submit

,is_final,pre_score
user_id,,
1,1,10000
4,37,440000
5,3,30000
6,85,890000
11,9,190000
...,...,...
1860,28,290000
1862,4,40000
1864,28,300000


In [ ]:
# Cat dataframe
new_data_frame = pd.concat([tries, isfinal_submit], axis=1)
new_data_frame

,tries,is_final,pre_score
1,2,1.0,10000.0
4,54,37.0,440000.0
5,6,3.0,30000.0
6,130,85.0,890000.0
7,5,NaN,NaN
...,...,...,...
1860,76,28.0,290000.0
1862,12,4.0,40000.0
1864,187,28.0,300000.0
1865,99,10.0,100000.0


In [ ]:
new_data_frame.rename(columns={'is_final':'solved', 'pre_score':'total_score'}, inplace=True)
new_data_frame['user_id'] = new_data_frame.index
first_column = new_data_frame.pop('user_id')
new_data_frame.insert(0, 'user_id', first_column)
new_data_frame

,user_id,tries,solved,total_score
1,1,2,1.0,10000.0
4,4,54,37.0,440000.0
5,5,6,3.0,30000.0
6,6,130,85.0,890000.0
7,7,5,NaN,NaN
...,...,...,...,...
1860,1860,76,28.0,290000.0
1862,1862,12,4.0,40000.0
1864,1864,187,28.0,300000.0
1865,1865,99,10.0,100000.0


In [ ]:
score_CK = data_score[['id', 'CK']]
score_CK.set_index('id', inplace=True)
new_data = pd.concat([new_data_frame, score_CK], axis=1)
new_data['user_id'] = new_data.index

,1,4,5,6,7,11,14,15,20,22,24,27,28,29,30,31,34,35,36,37,40,41
user_id,1,4,5,6,7,11,14,15,20,22,24,27,28,29,30,31,34,35,36,37,40,41
tries,2,54,6,130,5,50,22,101,13,NaN,164,13,NaN,199,NaN,13,218,297,NaN,312,664,110
solved,1,37,3,85,NaN,9,6,45,12,NaN,72,3,NaN,72,NaN,3,81,145,NaN,93,207,61
total_score,10000,440000,30000,890000,NaN,190000,70000,560000,120000,NaN,800000,40000,NaN,750000,NaN,30000,810000,1.52e+06,NaN,1e+06,2.27e+06,640000
CK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,9.5,10,9.5,8.5,9,NaN,6.5,4.5,7,7.5,8,8


In [ ]:
new_data

,user_id,tries,solved,total_score,CK
1,1,2.0,1.0,10000.0,NaN
4,4,54.0,37.0,440000.0,NaN
5,5,6.0,3.0,30000.0,NaN
6,6,130.0,85.0,890000.0,NaN
7,7,5.0,NaN,NaN,NaN
...,...,...,...,...,...
1862,1862,12.0,4.0,40000.0,7.5
1864,1864,187.0,28.0,300000.0,NaN
1865,1865,99.0,10.0,100000.0,7.5
1866,1866,NaN,NaN,NaN,5


In [ ]:
#find NaN values and replace it with 0
def count_missing_values(data):
  num_of_miss_data = data.isnull().sum()
  missing_data = pd.concat([num_of_miss_data], axis=1, keys = ['Number of missing value'])
  return missing_data
count_missing_values(new_data)


,Number of missing value
user_id,0
tries,122
solved,142
total_score,142
CK,307


In [ ]:
new_data['tries'] = new_data['tries'].interpolate(method = 'pad', limit = 3)
new_data['tries'] = new_data['tries'].fillna(int(np.mean(new_data['tries'])))

new_data['solved'] = new_data['solved'].interpolate(method = 'pad', limit = 3)
new_data['solved'] = new_data['solved'].fillna(int(np.mean(new_data['solved'])))

new_data['total_score'] = new_data['total_score'].interpolate(method = 'pad', limit = 3)
new_data['total_score'] = new_data['total_score'].fillna(int(np.mean(new_data['total_score'])))

new_data['CK'] = new_data['CK'].interpolate(method = 'pad', limit = 3)
new_data['CK'] = pd.to_numeric(new_data['CK'], errors='coerce')
new_data['CK'] = new_data['CK'].fillna(int(np.mean(new_data['CK'])))

In [ ]:
#find NaN values and replace it with 0
def count_missing_values(data):
  num_of_miss_data = data.isnull().sum()
  missing_data = pd.concat([num_of_miss_data], axis=1, keys = ['Number of missing value'])
  return missing_data
count_missing_values(new_data)

,Number of missing value
user_id,0
tries,0
solved,0
total_score,0
CK,0


In [ ]:
new_data.head(30).T

,1,4,5,6,7,11,14,15,20,22,24,27,28,29,30,31,34,35,36,37,40,41,42,43,44,45,47,48,49,51
user_id,1.0,4.0,5.0,6.0,7.0,11.0,14.0,15.0,20.0,22.0,24.0,27.0,28.0,29.0,30.0,31.0,34.0,35.0,36.0,37.0,40.0,41.0,42.0,43.0,44.0,45.0,47.0,48.0,49.0,51.0
tries,2.0,54.0,6.0,130.0,5.0,50.0,22.0,101.0,13.0,13.0,164.0,13.0,13.0,199.0,199.0,13.0,218.0,297.0,297.0,312.0,664.0,110.0,324.0,448.0,448.0,161.0,186.0,26.0,313.0,86.0
solved,1.0,37.0,3.0,85.0,85.0,9.0,6.0,45.0,12.0,12.0,72.0,3.0,3.0,72.0,72.0,3.0,81.0,145.0,145.0,93.0,207.0,61.0,191.0,148.0,148.0,55.0,66.0,19.0,97.0,38.0
total_score,10000.0,440000.0,30000.0,890000.0,890000.0,190000.0,70000.0,560000.0,120000.0,120000.0,800000.0,40000.0,40000.0,750000.0,750000.0,30000.0,810000.0,1520000.0,1520000.0,1000000.0,2270000.0,640000.0,1920000.0,1750000.0,1750000.0,560000.0,700000.0,190000.0,1010000.0,400000.0
CK,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.5,9.5,10.0,9.5,8.5,9.0,9.0,6.5,4.5,7.0,7.5,8.0,8.0,7.5,6.0,5.0,7.5,4.0,3.0,5.5,5.5


Spliting model

In [ ]:
X = new_data.iloc[:,new_data.columns!='CK']
y = new_data.iloc[:,-1]
y_test
predicted

array([6.40802194, 6.00088791, 5.57552433, 7.63789509, 5.87417839,
       5.68855251, 6.09022949, 6.22167587, 6.21515336, 6.06038386,
       5.98332129, 5.48656574, 6.9378983 , 6.34773661, 6.07458509,
       5.86594107, 6.8883672 , 6.47679919, 6.3497647 , 6.08803529,
       7.48706083, 6.8111453 , 6.26342831, 5.9367028 , 4.97883909,
       6.5828013 , 6.04717054, 5.69859047, 7.82146757, 6.23692912,
       6.45506377, 5.74488174, 5.98682362, 6.18716601, 6.94825528,
       6.15504272, 6.42806429, 6.60859075, 6.11691929, 5.63654837,
       5.56477887, 6.44138538, 6.47025906, 5.79327203, 7.54272931,
       5.96483649, 6.38325571, 6.81036403, 5.74706432, 6.23686779,
       5.95753889, 5.91950084, 6.31186639, 6.04229137, 6.00126138,
       7.02810536, 5.87404835, 5.73520826, 6.26634182, 5.94399486,
       6.01090425, 5.09477201, 6.08410948, 5.90831562, 6.1687451 ,
       5.66906639, 5.90869491, 6.27464249, 6.05846924, 5.97293693,
       6.04678143, 5.87529562, 6.11253572, 5.93870595, 6.31719

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
lr_model = LinearRegression().fit(x_train, y_train)
predicted = lr_model.predict(x_test)
# #print(lr_model.score(x_test, y_test))
# acc = accuracy_score(y_test, predicted)